In [1]:
from HelmholtzSolver2D import *
from ExampleBoundaries import Square

# Test Problem 1
# Dirichlet boundary condition with phi chose to be the same
# as a point-source at the center of the square.
#
solver = HelmholtzSolver2D(*(Square()))

boundaryCondition = BoundaryCondition(solver.aElement.shape[0])
boundaryCondition.alpha.fill(1.0)
boundaryCondition.beta.fill(0.0)

frequency = 400.0 # frequency [Hz]
k = frequencyToWavenumber(frequency)

centerSquare = np.array([0.05, 0.05], dtype=np.float32)
for i in range(solver.aCenters.shape[0]):
    r = norm(solver.aCenters[i, :] - centerSquare)
    boundaryCondition.f[i] = 1j * hankel1(0, k*r) / 4.0

boundaryIncidence = BoundaryIncidence(solver.aElement.shape[0])
boundaryIncidence.phi.fill(0.0)
boundaryIncidence.v.fill(0.0)

exteriorPoints = np.array([[0.0000,  0.1500],
                           [0.0500,  0.1500],
                           [0.1000,  0.1500],
                           [0.0500, -0.1000]], dtype=np.float32)

exteriorIncidentPhi = np.zeros(exteriorPoints.shape[0], dtype=np.complex64)

boundarySolution = solver.solveExteriorBoundary(k, boundaryCondition, boundaryIncidence)
exteriorPhi = solver.solveExterior(boundarySolution, exteriorIncidentPhi, exteriorPoints)
print "\n\nTest Problem 1"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, exteriorPhi)

# Test Problem 2
# Neumann boundary condition with v chosen to be the same
# as a point-source at the center of the square.
#
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)

centerSquare = np.array([0.05, 0.05], dtype=np.float32)
for i in range(solver.aCenters.shape[0]):
    x = solver.aCenters[i, :]
    r = x - centerSquare
    R = norm(r)
    if x[0] <= 1e-7:
        drbdn = -r[0]/R
    elif x[0] >= 0.1 - 1e-7:
        drbdn = r[0]/R
    elif x[1] <= 1e-7:
        drbdn = -r[1]/R
    elif x[1] >= 0.1 - 1e-7:
        drbdn = r[1]/R
    else:
        assert False, "r = ({}, {}) not properly handled.".format(r[0], r[1])
    boundaryCondition.f[i] = -1j * k * drbdn * hankel1(1, k*R) / 4.0

boundarySolution = solver.solveExteriorBoundary(k, boundaryCondition, boundaryIncidence)
exteriorPhi = solver.solveExterior(boundarySolution, exteriorIncidentPhi, exteriorPoints)
print "\n\nTest Problem 2"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, exteriorPhi)

# Test Problem 3
# Neumann boundary condition with v=0 on the full boundary.
# The 400Hz field in this case is generated by a point source
# at (0.5, 0.25)
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)
boundaryCondition.f.fill(0.0)

sourcePoint = np.array([0.5, 0.25], dtype=np.float32)
print "sourcePoint = ", sourcePoint
for i in range(solver.aCenters.shape[0]):
    x = solver.aCenters[i, :]
    r = x - sourcePoint
    R = norm(r)
    boundaryIncidence.phi[i] = 1j * hankel1(0, k*R) / 4.0
    if x[0] <= 1e-7:
        drbdn = -r[0]/R
    elif x[0] >= 0.1 - 1e-7:
        drbdn = r[0]/R
    elif x[1] <= 1e-7:
        drbdn = -r[1]/R
    elif x[1] >= 0.1 - 1e-7:
        drbdn = r[1]/R
    else:
        assert False, "r = ({}, {}) not properly handled.".format(r[0], r[1])
    boundaryIncidence.v[i] = -1j * k * drbdn * hankel1(1, k*R) / 4.0

boundarySolution = solver.solveExteriorBoundary(k, boundaryCondition, boundaryIncidence)

for i in range(exteriorPoints.shape[0]):
    R = norm(exteriorPoints[i, :] - sourcePoint)
    exteriorIncidentPhi[i] = 1j * hankel1(0, k*R) / 4.0
    
exteriorPhi = solver.solveExterior(boundarySolution, exteriorIncidentPhi, exteriorPoints)
print "\n\nTest Problem 3"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, exteriorPhi)



Test Problem 1

Density of medium:      1.205 kg/m^3
Speed of sound:         344.0 m/s
Wavenumber (Frequency): 7.30602942695 (400.0 Hz)

index          Potential                   Pressure                    Velocity              Intensity
    1   1.1657e-01+  2.3549e-01i   -7.1318e+02+  3.5303e+02i   -1.8591e+00+ -2.2398e-01i     6.2340e+02
    2   1.3824e-01+  2.3854e-01i   -7.2240e+02+  4.1865e+02i   -2.6194e+00+ -3.1019e-01i     8.8120e+02
    3   1.5594e-01+  2.4058e-01i   -7.2859e+02+  4.7227e+02i   -3.1497e+00+ -3.2510e-01i     1.0706e+03
    4   1.6620e-01+  2.4160e-01i   -7.3169e+02+  5.0335e+02i   -3.4997e+00+ -3.2924e-01i     1.1975e+03
    5   1.6620e-01+  2.4160e-01i   -7.3169e+02+  5.0335e+02i   -3.4997e+00+ -3.2927e-01i     1.1975e+03
    6   1.5594e-01+  2.4058e-01i   -7.2859e+02+  4.7227e+02i   -3.1499e+00+ -3.2572e-01i     1.0706e+03
    7   1.3824e-01+  2.3854e-01i   -7.2240e+02+  4.1865e+02i   -2.6195e+00+ -3.1064e-01i     8.8114e+02
    8   1.1657e-01+  2.3549e-0

Copyright (C) 2017 Frank Jargstorff

This file is part of the AcousticBEM library.

AcousticBEM is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

AcousticBEM is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with AcousticBEM. If not, see http://www.gnu.org/licenses/.